In [27]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

In [28]:
import sys
sys.setrecursionlimit(10000)
import uuid
import copy
from numpy import errstate

In [29]:
# def matrix_mul(a,b):
#   print("CEK", a[0][1].value)
#   c = np.array([[VarValue(0,varname=str(uuid.uuid4())) for _ in range(b.shape[1])] for _ in range(a.shape[0])])
#   for i in range(len(a)):
#     for j in range(len(b[0])):
#       for k in range(len(b)):
#         print("a", a[i][k].varname)
#         print("b", b[k][j].varname)
#         d = a[i][k] * b[k][j]
#         c[i][j] = c[i][j] + d
#   return c

In [30]:
import math
import uuid
from numpy import errstate

class VarValue:
    def __init__(self, value, children=(), varname=''):
        self.varname = varname
        self.value = value
        self.derivative_to = {}
        self.children = children

    def __chain_rule(self, dSelfdx, child):
        # print("Child of", self.varname,":",child.varname)
        if(child.varname[:5] != 'const'):
            for grandchild_varname in child.derivative_to:
                # print("Grandchild",grandchild_varname)
                # print("Grandchild children", len(grandchild.children))
                if(grandchild_varname != 'const'):# and (len(grandchild.children) == 0):
                    # print("Beneran masuk", grandchild_varname)
                    # print("Self:", self.varname)
                    if(grandchild_varname in self.derivative_to):
                        # print("grandchild", grandchild_varname)
                        self.derivative_to[grandchild_varname] += dSelfdx * child.derivative_to[grandchild_varname]
                    else:
                        # print("grandchild", grandchild_varname)
                        self.derivative_to[grandchild_varname] = dSelfdx * child.derivative_to[grandchild_varname]
            if(len(self.derivative_to) == 0):
              raise ValueError(self.varname, child.varname)

    def relu(self):
        out = VarValue(max(0,self.value), children=(self,), varname='out_relu_'+str(uuid.uuid4()))
        # print("Out is", out.varname)
        if(self.varname[:5] != 'const'):
            dodx = 0 if self.value <= 0 else 1
            if(len(self.children) == 0):
                out.derivative_to[self.varname] = dodx
            else:
                out.__chain_rule(dodx, self)
        return out

    def ln(self):
        out = VarValue(math.log(self.value), children=(self,), varname='out_ln_'+str(uuid.uuid4()))
        # print("Out is", out.varname)
        if(self.varname[:5] != 'const'):
            dodx = 1/(self.value)
            if(len(self.children) == 0):
                out.derivative_to[self.varname] = dodx
            else:
                out.__chain_rule(dodx, self)
        return out

    def __mul__(self, other):
        other = other if isinstance(other, VarValue) else VarValue(other, varname='const'+str(uuid.uuid4()))
        out = VarValue(self.value * other.value, children=(self, other), varname='out_mul_'+str(uuid.uuid4()))
        # print("Out is", out.varname)
        # print("Input1",self.varname, self.value)
        # if isinstance(self.value, VarValue):
            # print("Input1-VAR?!",self.varname, self.value.varname)
            # print("Input1-VAR?!",self.varname, self.value.value)
        # print("Input2",other.varname, other.value)
        if(self.varname[:5] != 'const'):
            dodx1 = other.value
            if(len(self.children) == 0):
                # print("Masuk sini", self.varname)
                out.derivative_to[self.varname] = dodx1
            else:
                # print("Chain Rule", self.varname)
                out.__chain_rule(dodx1, self)
        if(other.varname[:5] != 'const'):
            # print("Masuk sini", other.varname)
            dodx2 = self.value
            if(len(other.children) == 0):
                out.derivative_to[other.varname] = dodx2
            else:
                # print("Chain Rule", other.varname)
                out.__chain_rule(dodx2, other)
        return out

    def __add__(self, other):
        other = other if isinstance(other, VarValue) else VarValue(other, varname='const'+str(uuid.uuid4()))
        out = VarValue(self.value + other.value, children=(self, other), varname='out_add_'+str(uuid.uuid4()))
        # print("Out is", out.varname)
        if(self.varname[:5] != 'const'):
            dodx1 = 1
            if(len(self.children) == 0):
                # print("+new", self.varname)
                out.derivative_to[self.varname] = dodx1
            else:
                # print("+chain", self.varname)
                out.__chain_rule(dodx1, self)
        if(other.varname[:5] != 'const'):
            dodx2 = 1
            if(len(other.children) == 0):
                # print("+new", other.varname)
                out.derivative_to[other.varname] = dodx2
            else:
                # print("+chain", other.varname)
                out.__chain_rule(dodx2, other)
        return out

    def __sub__(self, other):
        return self + (-other)

    def __truediv__(self, other):
        return self * other**-1

    def __neg__(self):
        return self * -1

    def __pow__(self, other):
        other = other if isinstance(other, VarValue) else VarValue(other, varname='const'+str(uuid.uuid4()))
        try:
            with errstate(over='raise', invalid='raise'):
                result = float(self.value) ** float(other.value)
        except (OverflowError, FloatingPointError):
            if abs(float(self.value)) > 1:
                result = math.inf
            else:
                result = 0

        out = VarValue(result, children=(self, other), varname='out_pow_'+str(uuid.uuid4()))
        # out = VarValue(self.value**other.value, children=(self, other), varname='out_pow_'+str(uuid.uuid4()))
        # print("Out is", out.varname)
        if(self.varname[:5] != 'const'):
            dodx = other.value * self.value**(other.value-1)
            if(len(self.children) == 0):
                out.derivative_to[self.varname] = dodx
            else:
                out.__chain_rule(dodx, self)
        return out

    def __rmul__(self, other):
        return self * other

    def __radd__(self, other):
        return self + other

    def __rsub__(self, other):
        return other + (-self)

    def __rpow__(self, other):
        other = other if isinstance(other, VarValue) else VarValue(other, varname='const'+str(uuid.uuid4()))
        out = VarValue(other.value**self.value, children=(self, other), varname='out_rpow_'+str(uuid.uuid4()))
        # print("Out is", out.varname)
        if self.varname:
            dodx = other.value**self.value * math.log(other.value)
            if(len(self.children) == 0):
                out.derivative_to[self.varname] = dodx
            else:
                out.__chain_rule(dodx, self)
        return out

    def __rtruediv__(self, other):
        return other * self**-1

    # Equality
    def __eq__(self, other):
        if isinstance(other, VarValue):
            return self.varname == other.varname
        return self.varname == other

    # Inequality
    def __ne__(self, other):
        return not self.__eq__(other)

    # Less than
    def __lt__(self, other):
        if isinstance(other, VarValue):
            return self.value < other.value
        return self.value < other

    # Less than or equal
    def __le__(self, other):
        if isinstance(other, VarValue):
            return self.value <= other.value
        return self.value <= other

    # Greater than
    def __gt__(self, other):
        if isinstance(other, VarValue):
            return self.value > other.value
        return self.value > other

    # Greater than or equal
    def __ge__(self, other):
        if isinstance(other, VarValue):
            return self.value >= other.value
        return self.value >= other

In [31]:
import math
import numpy as np
import uuid

class Layer:
    def __init__(self,n_neurons=3, init='zero', activation='relu'):
        self.n_neurons = n_neurons
        self.current_input_batch = None
        self.init = init    # zero/uniform/normal - harusnya gaperlu disini, ini di layer langsung harusnya
        self.weights = None
        self.biases = None
        self.activation = activation    # linear/relu/sigmoid/tanh/softmax/binary_step/leaky_relu/prelu/elu/swish(ini)/gelu(ini)
        self.learning_rate = None

        self.net = None
        self.out = None

    def __update_weights_dEdW(self, dEdW):
        self.weights -= self.learning_rate*dEdW

    def __update_biases_dEdB(self, dEdB):
        self.biases -= self.learning_rate*dEdB

    def __update_weights_err_term(self, err_term):
        for input in self.current_input_batch:
            for i in self.weights:
                for j in i:
                    self.weights += self.learning_rate*err_term[j]*self.input[i]

    def __update_biases_err_term(self, err_term):
        for i in self.weights:
            for j in i:
                self.weights += self.learning_rate*err_term[j]*1

    def forward(self, current_input_batch):
        # print("KKCEK", current_input_batch[0][1].value)
        self.current_input_batch = current_input_batch

        if(self.weights is None):
            if(self.init == 'zero'):
                # print("len",len(self.current_input_batch[0]))
                self.weights = np.array([[VarValue(0,varname='w_'+str(uuid.uuid4())) for _ in range(self.n_neurons)] for _ in range(len(self.current_input_batch[0]))])
                self.biases = np.array([VarValue(0,varname='b_'+str(uuid.uuid4())) for _ in range(self.n_neurons)])
            elif(self.init == 'uniform'):
                # TODO: Initialization uniform
                pass
            elif(self.init == 'normal'):
                # TODO: Initialization normal
                pass

        self.net = (np.dot(self.current_input_batch, self.weights)) + self.biases
        # print(self.weights[0][0].varname)
        # print(self.current_input_batch.shape)
        # print(self.weights.shape)
        # print("NET")
        # self.net = (self.current_input_batch @ self.weights) + self.biases
        # print("LALCEK", self.current_input_batch[0][1].value)
        # self.net = matrix_mul(self.current_input_batch, self.weights) + self.biases
        # raise ValueError("STOP YE")

        if(self.activation == 'linear'):
            self.out = self.net

        elif(self.activation == 'relu'):
            self.out = np.array([[net.relu() for net in row] for row in self.net])

        elif(self.activation == 'sigmoid'):
            # self.out = 1/(1+(math.exp(1))**(-self.net))
            # low_net = np.array([[VarValue(0, varname=str(uuid.uuid4())) for j in i] for i in self.net ])
            # high_net = np.array([[VarValue(0, varname=str(uuid.uuid4())) for j in i] for i in self.net ])
            # for i in range(len(self.net)):
            #     for j in range(len(self.net[0])):
            #         low = VarValue(-500, varname='low'+str(uuid.uuid4()))
            #         low.derivative_to = {k: v for k, v in self.net[i][j].derivative_to.items()}
            #         low.children = copy.deepcopy(self.net[i][j].children)
            #         high = VarValue(500, varname='high'+str(uuid.uuid4()))
            #         high.derivative_to = {k: v for k, v in self.net[i][j].derivative_to.items()}
            #         high.children = copy.deepcopy(self.net[i][j].children)

            #         low_net[i][j] = low
            #         high_net[i][j] = high
            # self.out = 1 / (1 + (math.exp(1))**(-np.clip(self.net, low, high)))

            for i in self.net:
                for j in i:
                    j.value = np.clip(j.value, -500, 500)
            self.out = 1 / (1 + (math.exp(1))**(-self.net))


        elif(self.activation == 'tanh'):
            self.out = (math.exp(1)**self.net - math.exp(1)**(-self.net))/(math.exp(1)**self.net + math.exp(1)**(-self.net))

        # if type(self.out[0][0]) is float:
        #     print(self.activation,self.out)
        #     print(self.net[0][0])

    def backward(self, sum_err_term_k_w=None, err=None, method='instant_deriv'):
        if(method == 'instant_deriv'):
            # Versi pake dE/dW langsung - Tested
            # print("w_name_old")
            # for neuron_w in self.weights:
            #     for w in neuron_w:
            #         print(w.varname)
            dEdW = np.array([[ err.derivative_to[neuron_w.varname] for neuron_w in weights] for weights in self.weights])
            dEdB = np.array([ err.derivative_to[bias.varname] for bias in self.biases])

            self.__update_weights_dEdW(dEdW)
            self.__update_biases_dEdB(dEdB)
            # print("w_name_new", self.weights[0][0].varname)
            # raise ValueError("KAKA")
        else:
            # Versi pake Error Term - Not Tested Yet
            if(err):
                # Output Layer
                err_term = np.array([err.derivative_to[net.varname] for net in self.net])
            else:
                # Hidden Layer
                err_term = sum_err_term_k_w
                pass
            # sum_err_term_w_current = np.dot(self.weights,err_term.T)
            sum_err_term_w_current = self.weights @ err_term.T
            self.__update_weights_err_term(err_term)
            self.__update_biases_err_term(err_term)
            return sum_err_term_w_current,

    # Ini harusnya gaperlu
    def clean_derivative(self):
        for input in self.current_input_batch:
            for x in input:
                # x.derivative_to = {}
                x.derivative_to.clear()
                x.children = ()

        for i in self.weights:
            for j in i:
                # j.derivative_to = {}
                j.derivative_to.clear()
                j.children = ()

        for b in self.biases:
            b.derivative_to ={}
            b.children = ()

        for i in self.net:
            for j in i:
                # j.derivative_to = {}
                j.derivative_to.clear()
                j.children = ()

        for i in self.out:
            for j in i:
                # j.derivative_to = {}
                j.derivative_to.clear()
                j.children = ()

        # self.net = None
        # self.out = None

In [32]:
import uuid
import numpy as np
import time
from sklearn.preprocessing import OneHotEncoder

class FFNN:
    def __init__(self, loss='mse', batch_size=1, learning_rate=0.01, epochs=20, verbose=0):
        self.loss = loss    # mse/bce/cce
        self.batch_size=batch_size
        self.learning_rate=learning_rate
        self.epochs=epochs
        self.verbose=verbose
        self.layers = None
        self.weights = []
        self.bias = []
        self.x = None
        self.y = None
        self.onehot_encoder = OneHotEncoder(categories='auto')

    def __loss(self, out, target):
        if(out.shape != target.shape):
            print(out.shape)
            print(target.shape)
            raise ValueError("Shape not match")
        if self.loss == 'mse':
            mse = (1/target.shape[1])*np.square(target - out)
            return np.sum(mse)
        elif self.loss == 'cce':
            epsilon = 1e-15
            out = np.clip(out, epsilon, 1 - epsilon)
            return -np.sum(target * np.log(out)) / self.batch_size

    def build_layers(self, *layers: Layer):
        self.layers = layers
        for layer in self.layers:
            layer.learning_rate = self.learning_rate

    def fit(self, x, y):
        self.x = x
        self.y = self.onehot_encoder.fit_transform(y.reshape(-1, 1))
        total_batch = (len(x)+self.batch_size-1)//self.batch_size
        start_global = time.time()
        for epoch in range(self.epochs):
            print(f"Epoch {epoch+1}")
            start = time.time()
            count_batch = 1
            for i in range(total_batch):
                x_batch = self.x[i*self.batch_size:(i+1)*self.batch_size] if ((i+1) < total_batch) else self.x[i*self.batch_size:]
                y_batch = self.y[i*self.batch_size:(i+1)*self.batch_size] if ((i+1) < total_batch) else self.y[i*self.batch_size:]
                batch_input = x_batch
                for layer in self.layers:
                    # print("Forw")
                    layer.forward(batch_input)
                    # if(count_batch == 3):
                    #     for i in layer.out:
                    #         for j in i:
                    #           print(j.derivative_to)
                    batch_input = layer.out
                # print("Out")
                out = batch_input
                # print("Error")
                error = self.__loss(out, y_batch)
                count = 1

                # print("len_err_der", len(error.derivative_to))
                # if(count_batch == 3):
                #     for der_to in error.derivative_to:
                #         print("dEdsmth",der_to)

                for layer in self.layers[::-1]:
                    # print("Back")
                    # print("Batch-BackLay:",count_batch,count)
                    layer.backward(err=error)
                    count += 1
                count_batch += 1

                # Ini gaperlu harusnya gapapa
                for layer in self.layers:
                    layer.clean_derivative()

                # error.derivative_to.clear()
                # error.children = ()

            end = time.time()
            print("Dur", end-start)

        end_global = time.time()
        print("Dur Global", end_global-start_global)

    def predict(self, x_predict):
        batch_input = x_predict
        for layer in self.layers:
            layer.forward(batch_input)
            batch_input = layer.out
        out = batch_input
        return out

    def visualize(self):
        pass

    def show_weights_dist(self):
        pass

    def show_grad_dist(self):
        pass

    def save(self, filename):
        pass

    def load(self, filename):
        pass

In [33]:
X, y = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)
y = y.astype(np.uint8)

In [34]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=65500,
    train_size=10,
    random_state=42,
    stratify=y
)

In [35]:
X_train = X_train / 255.0
X_test = X_test / 255.0

In [36]:
X_train = np.array([[VarValue(x, varname='x_'+str(uuid.uuid4())) for x in row] for row in X_train])

In [37]:
model_ffnn = FFNN(
    loss='mse',
    batch_size=1,
    learning_rate=0.1,
    epochs=2,
    verbose=1,
)

In [ ]:
model_ffnn.build_layers(
    Layer(n_neurons=4, init='zero', activation='linear'),
    Layer(n_neurons=3, init='zero', activation='linear'),
    Layer(n_neurons=32, init='zero', activation='linear'),
    Layer(n_neurons=10, init='zero', activation='linear')
)

In [39]:
model_ffnn.fit(X_train, y_train)

Epoch 1
Dur 26.113123893737793
Epoch 2
Dur 20.67859411239624
Dur Global 46.79234743118286


In [40]:
for layer in model_ffnn.layers:
    print("=====Layer======")
    for i in layer.weights:
        print("==Neuron==")
        for j in i:
            print(j.varname)

=====Layer======
==Neuron==
out_add_58ec6ed1-08dd-44e3-adc9-64254a363e96
out_add_c1dcfbbe-6730-4fca-a8a8-16824bc0d707
out_add_6a71adde-990a-4dde-80a3-0abdb933869d
out_add_c9a3abc6-b7a7-48f3-857f-0fb998d9dcd8
out_add_5d934ece-3b59-4d0b-8d42-7d22fd9370a2
out_add_e5c2e6e0-3449-49f7-9c0a-034aea7ceda6
out_add_99d756a8-d216-4154-bc46-dbd8314a0a3d
out_add_94e241b5-a48c-47bc-a263-fd3e906241f5
out_add_58498740-8d3b-4f85-a246-d30cc109158d
out_add_129c149f-b9cc-4841-a8e0-0a6582107b3b
==Neuron==
out_add_7801f99d-386f-4af2-bc30-4dbeb998dd01
out_add_b63b94f9-de35-41d1-aa84-55a80d2d6394
out_add_1e077396-83a1-4962-a563-63ca559714ef
out_add_66b605ae-93fd-42ce-9929-e9acd0afb97e
out_add_8c6931dd-a196-44a9-8f60-ffbc657cfb62
out_add_2812889e-bd01-4e6b-a461-53761da11cef
out_add_0b5b2eef-e649-4d3f-99e8-3ccff94fdbba
out_add_5026c7e1-60f8-416a-baa1-1ba978790379
out_add_651cb279-5b19-4470-bd82-0e904822b6c7
out_add_6959b1cb-e940-4241-93fd-d80771958933
==Neuron==
out_add_f48230e7-85d9-4b52-ac32-a1c3965a0b63
out_a

In [41]:
for layer in model_ffnn.layers:
    for i in layer.weights:
        for j in i:
            print(j.value)

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [49]:
y_pred = model_ffnn.predict(X_test[:10])
for h in y_pred:
    max_index = np.argmax(h)
    print(max_index)

9
3
9
9
9
9
7
7
9
3


In [46]:
model_ffnn.y.shape

(10, 10)

In [47]:
for h in y_test[:10].flatten():
    print(h)

9
6
5
9
6
9
8
8
1
2
